In [1]:
import numpy as np
import pandas as pd
import os
import glob

In [2]:
subdir = '../expenses'
globPattern = os.path.join(subdir, '*.csv')
columnNames = ['Date', 'Payee', 'Debit', 'Credit', 'Balance']

# Rename CSV files

In [3]:
for file in glob.iglob(globPattern):   
    df = pd.read_csv(file, names=columnNames, header=None)
    month = pd.Timestamp(df['Date'].iloc[0])
    newFile = os.path.join(subdir, f'accountactivity_{month:%Y-%m}.csv')
    os.rename(file, newFile)

# Process the data

In [4]:
df = []

csvFiles = glob.glob(globPattern)
csvFiles.sort()
for file in csvFiles:
    df.append(pd.read_csv(file, names=columnNames, header=None))
    
df = pd.concat(df).reset_index(drop=True)
df['Date'] = pd.to_datetime(df['Date'])
df['Amount'] = -df['Debit'].combine_first(-df['Credit'])

df = df[['Date', 'Payee', 'Amount']]

In [5]:
def categorizer(row) -> str:
    payee = row['Payee'].upper()
    categories = {
        'TOR HYD ELEC': 'Hydro',
        'TOR UTILITY': 'Water',
        'TORONTO TAX': 'Property Tax',
        'WAWANESA INS': 'Insurance',
        'ENBRIDGE': 'Heat'
    }
    for key, value in categories.items():
        if payee.startswith(key):
            return value
    return None

df['Category'] = df.apply(categorizer, axis=1)

# Analysis

In [7]:
nmonths = 16
start = pd.Timestamp.now() - pd.offsets.DateOffset(months=nmonths)
# cond = (df['Date'] >= start) & (df['Category'] != '')
group = df[cond].groupby('Category')

summary = group.sum().assign(
    Monthly = lambda x: x['Amount'] / nmonths,
    Yearly = lambda x: x['Monthly'] * 12,
)[['Yearly', 'Monthly']]

total = summary.sum()
total.name = 'Total'
summary = summary.append(total)

summary.style.format({
    'Yearly': '{:,.2f}',
    'Monthly': '{:,.2f}'
})

,Yearly,Monthly
Category,,
Heat,"-1,127.03",-93.92
Hydro,"-1,194.68",-99.56
Insurance,-860.22,-71.69
Property Tax,"-3,423.48",-285.29
Water,-541.19,-45.10
Total,"-7,146.60",-595.55


In [56]:
years = df['Date'].dt.year
months = df['Date'].dt.month
n = months.groupby(years).max().values

# summary = df.groupby(['Category', years]).sum().unstack().apply(lambda row: row / n,axis=1)
summary = df.groupby(['Category', years]).sum().unstack().div(n)
summary = summary.droplevel(0, axis=1)
summary.columns.name = None

summary.style.format('{:,.2f}')

,2017,2018,2019
Category,,,
Heat,-18.08,-100.86,-129.66
Hydro,-14.88,-113.14,-93.41
Insurance,-88.83,-95.58,nan
Property Tax,nan,-244.67,-380.77
Water,-15.93,-45.44,-44.26


In [43]:
months = df['Date'].dt.month
n = months.groupby(years).max()

In [37]:
months

0     2017-11-01
1     2017-11-01
2     2017-11-01
3     2017-11-01
4     2017-11-01
         ...    
154   2019-08-01
155   2019-08-01
156   2019-08-01
157   2019-08-01
158   2019-08-01
Name: Date, Length: 159, dtype: datetime64[ns]

In [7]:
summary.to_excel('home_expenses.xlsx')

# Playground

In [17]:
df.to_excel('./~$expenses.xlsx', index=False)